Training the model

In [1]:
from tensorflow.keras import layers
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from kapre.time_frequency import Melspectrogram
from kapre.augmentation import AdditiveNoise
from kapre.utils import Normalization2D
import tensorflow as tf
import os
import time

In [2]:
def Conv2D(N_CLASSES=6, SR=16000, DT=1.0):
    initializer = tf.keras.initializers.TruncatedNormal(seed=None)
    i = layers.Input(shape=(1, int(SR*DT)), name='input')
    x = BatchNormalization()(i)
    x = Melspectrogram(n_dft=512, n_hop=160,
                       padding='same', sr=SR, n_mels=128,
                       fmin=0.0, fmax=SR/2, power_melgram=2.0,
                       return_decibel_melgram=True, trainable_fb=False,
                       trainable_kernel=False,
                       name='melbands')(x)
    x = AdditiveNoise(power=10., random_gain=True, noise_type='white')(x)
    x = BatchNormalization()(x)
    x = layers.Conv2D(8, kernel_size=(7,7), activation='relu', padding='same',kernel_initializer=initializer, name='conv2d_relu')(x)
    x = layers.MaxPooling2D(pool_size=(2,2), padding='same',  name='max_pool_2d_1')(x)
    x = BatchNormalization()(x)
    x = layers.Conv2D(16, kernel_size=(5,5), activation='relu', padding='same', kernel_initializer=initializer, name='conv2d_relu_1')(x)
    x = layers.MaxPooling2D(pool_size=(2,2), padding='same', name='max_pool_2d_2')(x)
    x = BatchNormalization()(x)
    x = layers.Conv2D(32, kernel_size=(3,3), activation='relu', kernel_initializer=initializer, padding='same', name='conv2d_relu_2')(x)
    x = layers.MaxPooling2D(pool_size=(2,2), padding='same', name='max_pool_2d_3')(x)
    x = BatchNormalization()(x)
    x = layers.Conv2D(32, kernel_size=(3,3), activation='relu', kernel_initializer=initializer, padding='same', name='conv2d_relu_3')(x)
    x = layers.MaxPooling2D(pool_size=(2,2), padding='same', name='max_pool_2d_4')(x)
    x = BatchNormalization()(x)
    x = layers.Flatten(name='flatten')(x)
    x = layers.Dropout(rate=0.2)(x)
    x = layers.Dense(64, activation='relu', kernel_initializer=initializer, name='dense1')(x)
    x = BatchNormalization()(x)
    x = layers.Dense(64, activation='relu', kernel_initializer=initializer, name='dense2')(x)
    x = BatchNormalization()(x)
    o = layers.Dense(N_CLASSES, activation='softmax', kernel_initializer=initializer, name='softmax')(x)

    model = Model(inputs=i, outputs=o, name='2d_convolution')
    model.summary()
    initial_learning_rate = 0.04
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate,
        decay_steps=500,
        decay_rate=0.96,
        staircase=True)
    
    opt = tf.keras.optimizers.Adagrad(learning_rate=lr_schedule)
    model.compile(optimizer= opt,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

def Conv1D(N_CLASSES=6, SR=16000, DT=1.0):
    i = layers.Input(shape=(1, int(SR*DT)), name='input')
    x = Melspectrogram(n_dft=512, n_hop=160,
                       padding='same', sr=SR, n_mels=128,
                       fmin=0.0, fmax=SR/2, power_melgram=2.0,
                       return_decibel_melgram=True, trainable_fb=False,
                       trainable_kernel=False,
                       name='melbands')(i)
    x = Normalization2D(str_axis='batch', name='batch_norm')(x)
    x = layers.Permute((2,1,3), name='permute')(x)
    x = TimeDistributed(layers.Conv1D(8, kernel_size=(4), activation='tanh'), name='td_conv_1d_tanh')(x)
    x = layers.MaxPooling2D(pool_size=(2,2), name='max_pool_2d_1')(x)
    x = TimeDistributed(layers.Conv1D(16, kernel_size=(4), activation='relu'), name='td_conv_1d_relu_1')(x)
    x = layers.MaxPooling2D(pool_size=(2,2), name='max_pool_2d_2')(x)
    x = TimeDistributed(layers.Conv1D(32, kernel_size=(4), activation='relu'), name='td_conv_1d_relu_2')(x)
    x = layers.MaxPooling2D(pool_size=(2,2), name='max_pool_2d_3')(x)
    x = TimeDistributed(layers.Conv1D(64, kernel_size=(4), activation='relu'), name='td_conv_1d_relu_3')(x)
    x = layers.MaxPooling2D(pool_size=(2,2), name='max_pool_2d_4')(x)
    x = TimeDistributed(layers.Conv1D(128, kernel_size=(4), activation='relu'), name='td_conv_1d_relu_4')(x)
    x = layers.GlobalMaxPooling2D(name='global_max_pooling_2d')(x)
    x = layers.Dropout(rate=0.1, name='dropout')(x)
    x = layers.Dense(64, activation='relu', activity_regularizer=l2(0.001), name='dense')(x)
    o = layers.Dense(N_CLASSES, activation='softmax', name='softmax')(x)

    model = Model(inputs=i, outputs=o, name='1d_convolution')
    #model.summary()
    opt = tf.keras.optimizers.Adam(0.00001)
    model.compile(optimizer=opt,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model



def LSTM(N_CLASSES=6, SR=16000, DT=1.0):
    i = layers.Input(shape=(1, int(SR*DT)), name='input')
    x = Melspectrogram(n_dft=512, n_hop=160,
                       padding='same', sr=SR, n_mels=128,
                       fmin=0.0, fmax=SR/2, power_melgram=2.0,
                       return_decibel_melgram=True, trainable_fb=False,
                       trainable_kernel=False,
                       name='melbands')(i)
    x = Normalization2D(str_axis='batch', name='batch_norm')(x)
    x = layers.Permute((2,1,3), name='permute')(x)
    x = TimeDistributed(layers.Reshape((-1,)), name='reshape')(x)
    s = TimeDistributed(layers.Dense(64, activation='tanh'),
                        name='td_dense_tanh')(x)
    x = layers.Bidirectional(layers.LSTM(32, return_sequences=True),
                             name='bidirectional_lstm')(s)
    x = layers.concatenate([s, x], axis=2, name='skip_connection')
    x = layers.Dense(64, activation='relu', name='dense_1_relu')(x)
    x = layers.MaxPooling1D(name='max_pool_1d')(x)
    x = layers.Dense(32, activation='relu', name='dense_2_relu')(x)
    x = layers.Flatten(name='flatten')(x)
    x = layers.Dropout(rate=0.2, name='dropout')(x)
    x = layers.Dense(32, activation='relu',
                         activity_regularizer=l2(0.003),
                         name='dense_3_relu')(x)
    o = layers.Dense(N_CLASSES, activation='softmax', name='softmax')(x)

    model = Model(inputs=i, outputs=o, name='long_short_term_memory')
    
    opt = tf.keras.optimizers.Adam(0.0008)
    model.compile(optimizer= 'Adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint
from tensorflow.keras.utils import to_categorical
import os
from scipy.io import wavfile
import pandas as pd
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
#from models import Conv1D, Conv2D, LSTM
from tqdm import tqdm
from glob import glob
import argparse

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, wav_paths, labels, sr, dt, n_classes,
                 batch_size=32, shuffle=True):
        self.wav_paths = wav_paths
        self.labels = labels
        self.sr = sr
        self.dt = dt
        self.n_classes = n_classes
        self.batch_size = batch_size
        self.shuffle = True
        self.on_epoch_end()
    def __len__(self):
        return int(np.floor(8*len(self.wav_paths) / self.batch_size))


    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        #print(index)
        wav_paths = [self.wav_paths[k] for k in indexes]
        labels = [self.labels[k] for k in indexes]

        # generate a batch of time data
        X = np.empty((self.batch_size, 1, int(self.sr*self.dt)), dtype=np.int16)
        Y = np.empty((self.batch_size, self.n_classes), dtype=np.float32)

        for i, (path, label) in enumerate(zip(wav_paths, labels)):
            rate, wav = wavfile.read(path)
            #print(wav.shape)
            #print(path+str(label))
            sample_length = int(rate*self.dt)
                    # generate a batch of time data
            X = np.empty((self.batch_size, 1, int(self.sr*self.dt)), dtype=np.int16)
            Y = np.empty((self.batch_size, self.n_classes), dtype=np.float32)
            
            if sample_length < wav.shape[0]:
                rand_index = np.random.randint(0, wav.shape[0]-sample_length)
                #rand_index = 1
                sample = wav[rand_index : rand_index+sample_length]
                
            else:
                #rand_index = 1
                rand_index = np.random.randint(0, sample_length-wav.shape[0])
                sample = np.pad(wav, (rand_index, sample_length-wav.shape[0]-rand_index), 'constant', constant_values=0)
            #print(sample)
            #print(rand_index)
            #print(path+str(label))
               
            #print(sample.shape) 
            
            X[i,] = sample.reshape(1, -1)
            #print(X[i]) 
            Y[i,] = to_categorical(label, num_classes=self.n_classes)
            #time.sleep(0.1)
        return X, Y
        

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.wav_paths))
        if self.shuffle:
            np.random.shuffle(self.indexes)


def train(args):
    src_root = args.src_root
    sr = args.sample_rate
    dt = args.delta_time
    batch_size = args.batch_size
    model_type = args.model_type
    params = {'N_CLASSES':len(os.listdir(args.src_root)),
              'SR':sr,
              'DT':dt}
    models = {'conv1d':Conv1D(**params),
              'conv2d':Conv2D(**params),
              'lstm':  LSTM(**params)}
    assert model_type in models.keys(), '{} not an available model'.format(model_type)
    csv_path = os.path.join('logs', '{}_history.csv'.format(model_type))

    wav_paths = glob('{}/**'.format(src_root), recursive=True)
    wav_paths = [x.replace(os.sep, '/') for x in wav_paths if '.wav' in x]
    classes = sorted(os.listdir(args.src_root))
    le = LabelEncoder()
    le.fit(classes)
    labels = [os.path.split(x)[0].split('/')[-1] for x in wav_paths]
    labels = le.transform(labels)

    wav_train, wav_val, label_train, label_val = train_test_split(wav_paths,
                                                                  labels,
                                                                  test_size=0.2,
                                                                  random_state=0)

    assert len(label_train) >= args.batch_size, 'number of train samples must be >= batch_size'

    tg = DataGenerator(wav_train, label_train, sr, dt,
                       len(set(label_train)), batch_size=batch_size)
    vg = DataGenerator(wav_val, label_val, sr, dt,
                       len(set(label_val)), batch_size=batch_size)

    model = models[model_type]
    cp = ModelCheckpoint('models/{}.h5'.format(model_type), monitor='val_loss',
                         save_best_only=True, save_weights_only=False,
                         mode='auto', save_freq='epoch', verbose=1)
    csv_logger = CSVLogger(csv_path, append=False)
    model.fit(tg, validation_data=vg,
              epochs=80, verbose=1,
              callbacks=[csv_logger, cp])



In [156]:
if __name__ == '__main__':

    parser = argparse.ArgumentParser(description='Audio Classification Training')
    parser.add_argument('--model_type', type=str, default='conv2d',
                        help='model to run. i.e. conv1d, conv2d, lstm')
    parser.add_argument('--src_root', type=str, default='data/clean1',
                        help='directory of audio files in total duration')
    parser.add_argument('--batch_size', type=int, default=64,
                        help='batch size')
    parser.add_argument('--delta_time', '-dt', type=float, default=4,
                        help='time in seconds to sample audio')
    parser.add_argument('--sample_rate', '-sr', type=int, default=16000,
                        help='sample rate of clean audio')
    args, _ = parser.parse_known_args()

    train(args)

Model: "2d_convolution"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 1, 64000)]        0         
_________________________________________________________________
batch_normalization_429 (Bat (None, 1, 64000)          256000    
_________________________________________________________________
melbands (Melspectrogram)    (None, 128, 400, 1)       296064    
_________________________________________________________________
batch_normalization_430 (Bat (None, 128, 400, 1)       4         
_________________________________________________________________
conv2d_relu (Conv2D)         (None, 128, 400, 8)       400       
_________________________________________________________________
max_pool_2d_1 (MaxPooling2D) (None, 64, 200, 8)        0         
_________________________________________________________________
batch_normalization_431 (Bat (None, 64, 200, 8)     

KeyboardInterrupt: 

In [ ]:
rate, wav = wavfile.read("data/new/awake/awake_0_1.wav")
length = len(wav)/rate
print(length)
